In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [6]:
teams_url = 'http://www.espn.com/nba/teams'
players_url = ''

In [7]:
html = urllib.request.urlopen(teams_url)
soup = BeautifulSoup(html, 'lxml')

In [8]:
links_store = str(soup.find_all('a'))

pattern = re.compile(r'href="/nba/team/_/name/\w*/\w*[-]?\w*')
matches = pattern.finditer(links_store)

teams = set()
for match in matches: 
    teams.add(match.group(0))

abbr_name = []
for link in teams:
    temp = link.split('/')
    abbr_name.append([temp[5], temp[6]])

abbr_name

[['no', 'new-orleans'],
 ['ny', 'new-york'],
 ['lac', 'la-clippers'],
 ['min', 'minnesota-timberwolves'],
 ['atl', 'atlanta-hawks'],
 ['den', 'denver-nuggets'],
 ['cha', 'charlotte-hornets'],
 ['orl', 'orlando-magic'],
 ['chi', 'chicago-bulls'],
 ['bos', 'boston-celtics'],
 ['wsh', 'washington-wizards'],
 ['mem', 'memphis-grizzlies'],
 ['phi', 'philadelphia-76ers'],
 ['por', 'portland-trail'],
 ['mil', 'milwaukee-bucks'],
 ['mia', 'miami-heat'],
 ['gs', 'golden-state'],
 ['utah', 'utah-jazz'],
 ['okc', 'oklahoma-city'],
 ['cle', 'cleveland-cavaliers'],
 ['hou', 'houston-rockets'],
 ['phx', 'phoenix-suns'],
 ['lal', 'los-angeles'],
 ['bkn', 'brooklyn-nets'],
 ['dal', 'dallas-mavericks'],
 ['sa', 'san-antonio'],
 ['tor', 'toronto-raptors'],
 ['ind', 'indiana-pacers'],
 ['sac', 'sacramento-kings'],
 ['det', 'detroit-pistons']]

In [9]:
home_split_format = 'https://www.espn.in/nba/team/stats/_/name/{team_abbr}/season/{season_no}/seasontype/{season_type}/split/{game_type}'
# codes for season type [2:Regular Season, 3:Post Season]
# codes for game_type [33: Home, 34:Away]
# data is available from 2002 - 2020

In [92]:
# # scrape the data for home games
# game_type = 33
master_df = pd.DataFrame(columns=['PNAME', 'POS', 'TEAM',  'YEAR', 'TYPE', 'GP', 'GS', 'MIN', 'PTS', 'OR', 'DR', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'AST/TO', 'PER'])
year_list = range(2002, 2021)
for item in abbr_name:
    team_abbr = item[0]
    for year in year_list:
        season_no = year
        season_type = '2'
        game_types = ['33', '34']
        for game_type in game_types:
            time.sleep(3)
            test_split_link = 'https://www.espn.in/nba/team/stats/_/name/{team_abbr}/season/{season_no}/seasontype/{season_type}/split/{game_type}'.format(team_abbr=team_abbr, season_no=season_no, season_type=season_type, game_type=game_type)
            print(test_split_link)
            test_html = urllib.request.urlopen(test_split_link)
            test_soup = BeautifulSoup(test_html, 'lxml')
            players = []
            try:
                tables = test_soup.find_all('table')
                player_names = tables[0]
                rows = player_names.find_all('tr')
                # ignore first row
                for row in rows[1:]:
                    tds = row.find_all('td')[0]
                    spans = tds.find_all('span')[0]
                    try:
                        temp = [span.getText() for span in spans]
                        # remove trailing '*' after some of the position values
                        if len(temp) > 2:
                            temp = temp[:-1]
                        # removing leading spaces for position
                        temp[1] = temp[1].lstrip(' ')
                        players.append(temp)
                    except Exception as e:
                        pass

                all_rows = []
                tables = test_soup.find_all('table')
                player_stats = tables[1]
                rows = player_stats.find_all('tr')
                for row in rows[1:-1]:
                    tds = row.find_all('td')
                    vals = [td.getText() for td in tds]
                    all_rows.append(vals)

                concat_name_stats = []
                for idx in range(len(players)):
                    concat_name_stats.append(players[idx] + [team_abbr] + [season_no] + [game_type] +  all_rows[idx])

                temp_df = pd.DataFrame(concat_name_stats,columns=['PNAME', 'POS', 'TEAM',  'YEAR', 'TYPE', 'GP', 'GS', 'MIN', 'PTS', 'OR', 'DR', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'AST/TO', 'PER'])
                master_df = pd.concat([master_df, temp_df], axis=0)
            except Exception as e:
                pass


https://www.espn.in/nba/team/stats/_/name/no/season/2002/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/no/season/2002/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/no/season/2003/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/no/season/2003/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/no/season/2004/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/no/season/2004/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/no/season/2005/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/no/season/2005/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/no/season/2006/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/no/season/2006/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/no/season/2007/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/no/season/2007/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/no/season/

https://www.espn.in/nba/team/stats/_/name/lac/season/2016/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/lac/season/2016/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/lac/season/2017/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/lac/season/2017/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/lac/season/2018/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/lac/season/2018/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/lac/season/2019/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/lac/season/2019/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/lac/season/2020/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/lac/season/2020/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/min/season/2002/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/min/season/2002/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/den/season/2010/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/den/season/2011/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/den/season/2011/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/den/season/2012/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/den/season/2012/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/den/season/2013/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/den/season/2013/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/den/season/2014/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/den/season/2014/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/den/season/2015/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/den/season/2015/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/den/season/2016/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/chi/season/2005/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/chi/season/2005/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/chi/season/2006/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/chi/season/2006/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/chi/season/2007/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/chi/season/2007/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/chi/season/2008/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/chi/season/2008/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/chi/season/2009/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/chi/season/2009/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/chi/season/2010/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/chi/season/2010/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/wsh/season/2018/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/wsh/season/2019/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/wsh/season/2019/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/wsh/season/2020/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/wsh/season/2020/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/mem/season/2002/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/mem/season/2002/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/mem/season/2003/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/mem/season/2003/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/mem/season/2004/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/mem/season/2004/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/mem/season/2005/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/nam

...

https://www.espn.in/nba/team/stats/_/name/cle/season/2002/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/cle/season/2002/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/cle/season/2003/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/cle/season/2003/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/cle/season/2004/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/cle/season/2004/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/cle/season/2005/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/cle/season/2005/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/cle/season/2006/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/cle/season/2006/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/cle/season/2007/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/cle/season/2007/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/phx/season/2015/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2016/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2016/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2017/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2017/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2018/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2018/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2019/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2019/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/phx/season/2020/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/phx/season/2020/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/lal/season/2002/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/dal/season/2010/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/dal/season/2010/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/dal/season/2011/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/dal/season/2011/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/dal/season/2012/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/dal/season/2012/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/dal/season/2013/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/dal/season/2013/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/dal/season/2014/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/dal/season/2014/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/dal/season/2015/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/dal/season/2015/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/ind/season/2004/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/ind/season/2005/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/ind/season/2005/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/ind/season/2006/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/ind/season/2006/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/ind/season/2007/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/ind/season/2007/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/ind/season/2008/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/ind/season/2008/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/ind/season/2009/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/ind/season/2009/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/ind/season/2010/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/nam

https://www.espn.in/nba/team/stats/_/name/det/season/2018/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/det/season/2018/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/det/season/2019/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/det/season/2019/seasontype/2/split/34
https://www.espn.in/nba/team/stats/_/name/det/season/2020/seasontype/2/split/33
https://www.espn.in/nba/team/stats/_/name/det/season/2020/seasontype/2/split/34


In [93]:
master_df

,PNAME,POS,TEAM,YEAR,TYPE,GP,GS,MIN,PTS,OR,DR,REB,AST,STL,BLK,TO,PF,AST/TO,PER
0,Jamal Mashburn,SF,no,2003,33,41,40,40.4,22.4,0.7,5.6,6.3,5.7,1.0,0.3,2.9,2.3,2.0,0.00
1,David Wesley,SG,no,2003,33,36,36,38.1,17.6,0.7,1.8,2.5,4.0,1.6,0.2,1.8,2.5,2.2,0.00
2,Baron Davis,PG,no,2003,33,25,25,37.0,16.9,1.0,2.7,3.7,7.4,2.0,0.6,2.8,3.0,2.7,0.00
3,Jamaal Magloire,C,no,2003,33,41,41,31.5,11.5,3.7,6.3,10.0,1.5,0.7,1.6,1.9,3.2,0.8,0.00
4,P.J. Brown,PF,no,2003,33,39,39,32.9,11.3,2.7,5.7,8.4,1.9,0.9,1.0,1.3,2.6,1.4,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,Khyri Thomas,SG,det,2020,34,6,0,8.2,2.7,0.2,0.0,0.2,0.3,0.3,0.0,0.3,1.5,1.0,0.00
18,Donta Hall,PF,det,2020,34,4,0,12.0,1.5,1.5,2.3,3.8,0.5,0.3,0.3,0.5,2.3,1.0,0.00
19,Jordan Bone,PG,det,2020,34,3,0,4.7,1.3,0.0,0.0,0.0,0.3,0.3,0.0,0.3,1.0,1.0,0.00
20,Louis King,SF,det,2020,34,4,0,3.8,1.3,0.3,0.5,0.8,0.3,0.0,0.0,0.3,0.5,1.0,0.00


In [94]:
master_df.to_csv('home_away.csv', index=False)